# Course project


## **Основное**
- Дедлайн - 4 апреля 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. В решении должны быть отчетливо видна метрика на новом тестовом сете из файла retail_test1.csv, то есть вам нужно для всех юзеров из этого файла выдать выши рекомендации, и посчитать на actual покупках precision@5. 

**!! Мы не рассматриваем холодный старт для пользователя, все наши пользователя одинаковы во всех сетах, поэтому нужно позаботиться об их исключении из теста.**


**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

In [69]:
!pip install implicit==0.4.4

# Import libs

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

## Read data

In [71]:
PATH_DATA = "./data"

In [72]:
data = pd.read_csv(os.path.join(PATH_DATA,'retail_train.csv'))
item_features = pd.read_csv(os.path.join(PATH_DATA,'product.csv'))
user_features = pd.read_csv(os.path.join(PATH_DATA,'hh_demographic.csv'))

# Set global const

In [73]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'

# N = Neighbors
N_PREDICT = 50 

# Process features dataset

In [74]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

# Split dataset for train, eval, test

In [75]:
# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

In [76]:
# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                      (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]

# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [77]:
# сделаем объединенный сет данных для первого уровня (матчинга)
df_join_train_matcher = pd.concat([data_train_matcher, data_val_matcher])

In [78]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [79]:
print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329


In [80]:
# выше видим разброс по пользователям и товарам и дальше мы перейдем к warm-start (только известные пользователи)

In [81]:
data_val_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2104867,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,0.0,0.0
2107468,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,0.0,0.0


# Prefilter items

In [145]:
# from utils import prefilter_items

n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 5001 to 5000


# Make cold-start to warm-start

In [83]:
# ищем общих пользователей
common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

train_matcher
Shape: (861404, 13) Users: 2495 Items: 5001
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


# Init/train recommender

In [84]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

### Варианты, как получить кандидатов

Можно потом все эти варианты соединить в один

(!) Если модель рекомендует < N товаров, то рекомендации дополняются топ-популярными товарами до N

In [146]:
recommender.top_purchases.head(3)  # Топ покупок каждого юзера

,user_id,item_id,quantity
182094,1479,977374,116
42966,358,5569230,115
199231,1609,6632283,112


In [147]:
recommender.overall_top_purchases[:3]  # Топ покупок по всему датасету

[1029743, 1106523, 5569230]

In [148]:
# Берем тестового юзера 2375

In [149]:
recommender.get_als_recommendations(2375, N=5)

[871756, 899624, 1044078, 1106523, 844179]

In [150]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [151]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 1044078, 999270, 934399, 1133312]

In [152]:
recommender.get_similar_users_recommendation(2375, N=5)

[1097398, 894360, 1088771, 977559, 1087411]

# Eval recall of matching

### Измеряем recall@k

Это будет в ДЗ: 

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [153]:
result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [154]:
%%time
# для понятности расписано все в строчку, без функций, ваша задача уметь оборачивать все это в функции
result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_items_recommendation(x, N=N_PREDICT))
result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_als_recommendations(x, N=N_PREDICT))

CPU times: user 2min 8s, sys: 5.35 s, total: 2min 14s
Wall time: 1min 24s


In [155]:
%%time
# result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL].apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

CPU times: user 18 µs, sys: 2 µs, total: 20 µs
Wall time: 19.1 µs


### Пример оборачивания

In [156]:
# # сырой и простой пример как можно обернуть в функцию
def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

In [157]:
evalRecall(result_eval_matcher, USER_COL, recommender.get_own_recommendations)

0.044119547395835505

In [158]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [159]:
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

### Recall@50 of matching

In [160]:
TOPK_RECALL = 50

In [161]:
sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

[('own_rec', 0.06525657038145175),
 ('als_rec', 0.04810321152133131),
 ('result', 0.044119547395835505),
 ('sim_item_rec', 0.03320483428552796)]

### Precision@5 of matching

In [99]:
TOPK_PRECISION = 5

In [100]:
sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

[('own_rec', 0.17712691771268974),
 ('als_rec', 0.12226871222687007),
 ('sim_item_rec', 0.05857740585774101)]

# Ranking part

### Обучаем модель 2-ого уровня на выбранных кандидатах

- Обучаем на data_train_ranking
- Обучаем *только* на выбранных кандидатах
- Я *для примера* сгенерирую топ-50 кадидиатов через get_own_recommendations
- (!) Если юзер купил < 50 товаров, то get_own_recommendations дополнит рекоммендации топ-популярными

In [101]:
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 

## Подготовка данных для трейна

In [162]:
# взяли пользователей из трейна для ранжирования
df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [163]:
# собираем кандитатов с первого этапа (matcher)
df_match_candidates['candidates'] = df_match_candidates[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

In [164]:
df_match_candidates.head(2)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."


In [165]:
# разворачиваем товары
df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items.name = 'item_id'

In [166]:
df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)

In [167]:
df_match_candidates.head(4)

,user_id,item_id
0,2070,1105426
0,2070,1097350
0,2070,879194
0,2070,948640


### Check warm start

In [168]:
print_stats_data(df_match_candidates, 'match_candidates')

match_candidates
Shape: (107550, 2) Users: 2151 Items: 4574


### Создаем трейн сет для ранжирования с учетом кандидатов с этапа 1 

In [169]:
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только покупки 

df_ranker_train.head()

,user_id,item_id,target
2104867,2070,1019940,1
2107468,2021,840361,1
2107469,2021,856060,1
2107470,2021,869344,1
2107471,2021,896862,1


In [170]:
df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

# чистим дубликаты
df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])

df_ranker_train['target'].fillna(0, inplace= True)

In [171]:
df_ranker_train.target.value_counts()

0.0    99177
1.0     7795
Name: target, dtype: int64

In [172]:
df_ranker_train.head(2)

,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0


(!) На каждого юзера 50 item_id-кандидатов

In [173]:
df_ranker_train['target'].mean()

0.07286953595333358

## Подготавливаем фичи для обучения модели

### Описательные фичи

In [174]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [175]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [176]:
df_ranker_train = df_ranker_train.merge(item_features, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


**Фичи user_id:**
    - Средний чек
    - Средняя сумма покупки 1 товара в каждой категории
    - Кол-во покупок в каждой категории
    - Частотность покупок раз/месяц
    - Долю покупок в выходные
    - Долю покупок утром/днем/вечером

**Фичи item_id**:
    - Кол-во покупок в неделю
    - Среднее ол-во покупок 1 товара в категории в неделю
    - (Кол-во покупок в неделю) / (Среднее ол-во покупок 1 товара в категории в неделю)
    - Цена (Можно посчитать из retil_train.csv)
    - Цена / Средняя цена товара в категории
    
**Фичи пары user_id - item_id**
    - (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)
    - (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)
    - (Кол-во покупок юзером конкретной категории в неделю) / (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

### Поведенческие фичи

##### Чтобы считать поведенческие фичи, нужно учесть все данные что были до data_val_ranker

In [177]:
# Находим средний чек для пользователя
mean_user_checks = df_join_train_matcher.fillna(value=0).groupby(['user_id'])['sales_value'].mean().reset_index()

mean_user_checks.rename(columns={'sales_value': 'mean_user_check'}, inplace=True)
mean_user_checks.head()

,user_id,mean_user_check
0,1,2.494884
1,2,2.783893
2,3,2.936759
3,4,3.987076
4,5,3.420502


In [178]:
# Находим средний чек для пользователя в неделю
mean_user_week_checks = \
    (df_join_train_matcher.fillna(value=0).groupby(['user_id'])['sales_value'].sum()
    /len(data['week_no'].unique())).reset_index()

mean_user_week_checks.rename(columns={'sales_value': 'mean_user_week_check'}, inplace=True)
mean_user_week_checks.head()

,user_id,mean_user_week_check
0,1,39.576737
1,2,19.194211
2,3,26.894526
3,4,12.632737
4,5,7.885158


In [179]:
# Средний чек товара в корзине, среднее количество товара в корзине
mean_basket_item_checks = df_join_train_matcher.fillna(value=0).groupby(['item_id']) \
        [['sales_value', 'quantity']].mean().reset_index()

mean_basket_item_checks.rename(columns={'sales_value': 'item_check', 'quantity': 'item_quantity'}, inplace=True)
mean_basket_item_checks.head()

,item_id,item_check,item_quantity
0,25671,6.98,2.0
1,26081,0.99,1.0
2,26093,1.59,1.0
3,26190,1.54,1.0
4,26355,1.98,2.0


In [180]:
# Цена товара
item_priсes = (df_join_train_matcher.fillna(value=0).groupby(['item_id']) \
        ['sales_value'].mean()/df_join_train_matcher.fillna(value=0).groupby(['item_id']) \
        ['quantity'].mean()).reset_index()


item_priсes.rename(columns={0: 'item_priсe'}, inplace=True)
item_priсes.head()

,item_id,item_priсe
0,25671,3.49
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,0.99


In [181]:
# (Средний чек на конкретной категории купленный юзером в неделю) / 
# (Средний чек купленный всеми юзерами конкретной категории в неделю)

user_item_week_ratios = (
      ( df_join_train_matcher.fillna(value=0).groupby(['item_id', 'user_id', 'week_no'])['sales_value'].mean() /
        len(df_join_train_matcher['week_no'].unique()) ) /
      ( df_join_train_matcher.fillna(value=1).groupby(['item_id', 'week_no'])['sales_value'].mean() /
        len(df_join_train_matcher['week_no'].unique()) )
).reset_index()

user_item_week_ratios.rename(columns={'sales_value': 'user_item_week_ratio'}, inplace=True)
# mean_basket_check.loc[mean_basket_check['sales_value'] > 1]
user_item_week_ratios.tail(2)

,item_id,week_no,user_id,user_item_week_ratio
2210411,17829232,91,1163,1.0
2210412,17829232,91,1596,1.0


In [182]:
# Средняя сумма покупки 1 товара, среднее количество товара в корзине пользователя
sum_sale_quantities = data.fillna(value=0).groupby(['user_id', 'item_id']) \
        [['sales_value', 'quantity']].mean().reset_index()

sum_sale_quantities.rename(columns={'sales_value': 'sum_sale', 'quantity': 'item_quantity'}, inplace=True)
sum_sale_quantities.head(10)

,user_id,item_id,sum_sale,item_quantity
0,1,819312,5.670000,1.000000
1,1,820165,1.750000,3.500000
2,1,821815,3.380000,2.000000
3,1,821867,0.690000,1.000000
4,1,823721,2.990000,1.000000
5,1,823990,6.700000,1.000000
6,1,825123,3.993333,1.333333
7,1,826695,2.690000,1.000000
8,1,827656,3.670000,1.000000
9,1,827671,1.490000,1.000000


In [183]:
# Средняя сумма покупки каждой категории
mean_user_item_checks = data.fillna(value=0).groupby(['item_id', 'user_id']) \
        [['sales_value']].mean().reset_index()

mean_user_item_checks.rename(columns={'sales_value': 'user_item_sales_value'}, inplace=True)
mean_user_item_checks.head()

,item_id,user_id,user_item_sales_value
0,25671,325,13.96
1,25671,358,3.49
2,25671,1228,3.49
3,26081,1675,0.99
4,26093,1032,1.59


In [184]:
#  (Средний чек на конкретной категории купленный юзером в неделю) /
# (Среднее кол-во покупок конкретной категории всеми юзерами в неделю)
user_item_week_check = (df_join_train_matcher.fillna(value=0).groupby(['item_id', 'user_id', 'week_no']) \
        ['sales_value'].mean()/df_join_train_matcher.fillna(value=1).groupby(['item_id', 'week_no']) \
        ['quantity'].mean()).reset_index()


user_item_week_check.rename(columns={0: 'user_item_week_checks'}, inplace=True)
user_item_week_check.head(2)

,item_id,week_no,user_id,user_item_week_checks
0,25671,23,1228,3.49
1,25671,36,358,3.49


In [185]:
df_ranker_train = df_ranker_train.merge(mean_user_checks, on='user_id', how='left')
df_ranker_train = df_ranker_train.merge(mean_user_week_checks, on=['user_id'], how='left')

df_ranker_train = df_ranker_train.merge(mean_basket_item_checks, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(item_priсes, on=['item_id'], how='left')

# df_ranker_train = df_ranker_train.merge(user_item_week_ratios, on=['item_id', 'user_id'], how='left')
df_ranker_train = df_ranker_train.merge(sum_sale_quantities, on=['item_id', 'user_id'], how='left')
df_ranker_train = df_ranker_train.merge(mean_user_item_checks, on=['user_id', 'item_id'], how='left')

# df_ranker_train = df_ranker_train.merge(user_item_week_check, on=['user_id', 'item_id'], how='left')

df_ranker_train.head(2)


,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,mean_user_check,mean_user_week_check,item_check,item_quantity_x,item_priсe,sum_sale,item_quantity_y,user_item_sales_value
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.883196,60.577474,4.473737,1.141414,3.919469,3.99,1.0,3.99
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.883196,60.577474,12.146275,1.058824,11.471481,NaN,NaN,NaN


In [186]:
df_join_train_matcher.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


## !!! Пока выполните нотбук без этих строк, потом вернитесь и запустите их, обучите ранкер и посмотрите на метрики с ранжированием

In [187]:
# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('sales_value').sum().rename('total_item_sales_value'), how='left',on=ITEM_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('total_quantity_value'), how='left',on=ITEM_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq'), how='left',on=ITEM_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq'), how='left',on=USER_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('sales_value').sum().rename('total_user_sales_value'), how='left',on=USER_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=ITEM_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_week')/df_join_train_matcher.week_no.nunique(), how='left',on=USER_COL)


# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg('quantity').sum().rename('item_quantity_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg('quantity').sum().rename('user_quantity_per_baskter')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=ITEM_COL).agg(USER_COL).count().rename('item_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=ITEM_COL)

# df_ranker_train = df_ranker_train.merge(df_join_train_matcher.groupby(by=USER_COL).agg(USER_COL).count().rename('user_freq_per_basket')/df_join_train_matcher.basket_id.nunique(), how='left',on=USER_COL)


In [188]:
df_ranker_train.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,mean_user_check,mean_user_week_check,item_check,item_quantity_x,item_priсe,sum_sale,item_quantity_y,user_item_sales_value
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.883196,60.577474,4.473737,1.141414,3.919469,3.990,1.0,3.990
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.883196,60.577474,12.146275,1.058824,11.471481,NaN,NaN,NaN
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.883196,60.577474,8.495870,1.173913,7.237222,10.785,1.5,10.785
3,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.883196,60.577474,7.345682,1.113636,6.596122,NaN,NaN,NaN
4,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.883196,60.577474,8.511321,1.113208,7.645763,10.785,1.5,10.785


In [189]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [190]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

## Обучение модели ранжирования

In [192]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=10,
                     n_estimators=500,
                     learning_rate=0.5,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)
train_preds = lgb.predict_proba(X_train)

In [193]:
df_ranker_predict = df_ranker_train.copy()

In [194]:
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

## Подведем итоги

    Мы обучили модель ранжирования на покупках из сета data_train_ranker и на кандитатах от own_recommendations, что является тренировочным сетом, и теперь наша задача предсказать и оценить именно на тестовом сете.

# Evaluation on test dataset

In [195]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


## Eval matching on test dataset

In [196]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 13.6 s, sys: 1.59 s, total: 15.2 s
Wall time: 16.1 s


In [197]:
# померяем precision только модели матчинга, чтобы понимать влияение ранжирования на метрики

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.1444117647058813)]

## Eval re-ranked matched result on test dataset
    Вспомним df_match_candidates сет, который был получен own_recommendations на юзерах, набор пользователей мы фиксировали и он одинаков, значи и прогноз одинаков, поэтому мы можем использовать этот датафрейм для переранжирования.
    

In [198]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [199]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

## Проверьте данные метрики с фичами и без (PS: должен быть прирост)

In [200]:
# смотрим на метрики выше и сравниваем что с ранжированием и без, добавляем фичи и то же смотрим
# в первом приближении метрики должны расти с использованием второго этапа

print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.22579634464751738)
('own_rec', 0.1444117647058813)


In [201]:
(0.22579634464751738/0.1444117647058813-1)*100

56.355920937182226

# Оценка на тесте для выполнения курсового проекта

In [206]:
df_test = pd.read_csv(os.path.join('retail_test1.csv'))

In [207]:
df_test = df_test[df_test.user_id.isin(common_users)]
df_test.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0
2,2070,41652857291,664,995242,5,9.10,311,-0.6,46,96,0.0,0.0


In [208]:
result_test = df_test.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [209]:
print_stats_data(df_test,'test')

test
Shape: (88665, 12) Users: 1883 Items: 20492


In [210]:
%%time
result_test['own_rec'] = result_test[USER_COL]. \
        apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 12.3 s, sys: 1.19 s, total: 13.5 s
Wall time: 14.3 s


In [211]:
sorted(calc_precision(result_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

[('own_rec', 0.1227827934147629)]

In [212]:
result_test['reranked_own_rec'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))

In [213]:
print(*sorted(calc_precision(result_test, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

('reranked_own_rec', 0.1760135135135118)
('own_rec', 0.1227827934147629)


In [214]:
result_test.head()

,user_id,actual,own_rec,reranked_own_rec
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[856942, 9297615, 5577022, 877391, 9655212, 88...","[9655212, 8091601, 9297615, 8293439, 983897]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[911974, 1076580, 1103898, 5567582, 1056620, 9...","[1103898, 1131634, 904236, 843418, 911974]"
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109...","[1092937, 1008714, 12132312, 1075979, 998206, ...",[]
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[13003092, 972416, 995598, 923600, 1138596, 10...","[1098844, 13115971, 900802, 13115375, 897088]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[998519, 894360, 7147142, 9338009, 896666, 939...","[1122358, 9338009, 870882, 1091793, 983316]"


In [215]:
(0.1760135135135118/0.1227827934147629-1)*100

43.353566585616264